<a href="https://colab.research.google.com/github/karen1203/LimpiezaCredicel/blob/main/Propuesta_uno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
credi= pd.read_excel("cuentas_credicel.xlsx")

In [18]:
credi

,folio,tag,folio_solicitud,fecha,marca,modelo,plazo,precio,enganche,descuento,...,fecha_pago_proximo,status_cuenta,puntos,riesgo,porc_enganche,porc_tasa,score_buro,razones_buro,semana_actual,codigo_postal
0,3,HDNAWQ3A,30,2021-11-12 19:00:05,ZTE,Blade A3 2020 RO B,26S,1949.0,780.0,0.0,...,NaT,NaN,NaN,NaN,NaN,0.00,0.0,0,0,0
1,4,M418YNR4,33,2021-11-12 19:15:58,MOTOROLA,LTE XT2097-12 E7I POWER Azul,13S,2999.0,1050.0,0.0,...,NaT,NaN,NaN,NaN,NaN,0.00,0.0,0,0,0
2,5,SZ7V3NZT,37,2021-11-13 14:13:08,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,NaT,NaN,NaN,Cancelado,0.00,0.00,0.0,0,0,NaN
3,6,9I08UIQZ,38,2021-11-13 15:45:39,ZTE,BLADE A3 2020 Gris,13S,1959.0,490.0,0.0,...,NaT,NaN,NaN,NaN,NaN,0.00,0.0,0,0,0
4,7,JMF4LKYU,44,2021-11-13 16:10:15,MOTOROLA,G10 B,39S,4299.0,1075.0,0.0,...,2022-03-12,Mora,0.0,0,0.00,0.00,0.0,NaN,NaN,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22730,22967,92KQVRIX,96502,2023-04-18 18:45:00,Motorola,Moto E13,26S,3349.0,297.0,0.0,...,2023-04-25,NaN,NaN,10,27.40,-1.13,0.0,0,"K0,J0,D8,E2",0
22731,22968,NWB2EA8J,96479,2023-04-18 18:46:53,Motorola,Moto E22i,26S,4100.0,1209.0,0.0,...,2023-04-25,NaN,NaN,9,19.00,7.48,0.0,0,"K0,J0,D8,E2",0
22732,22969,VDTRJ2W6,96442,2023-04-18 18:47:47,Samsung,A13,26S,4849.0,1152.0,0.0,...,2023-04-25,NaN,NaN,11,26.65,0.75,0.0,0,"K0,D7,F7,D8",0
22733,22970,PETLF56C,96536,2023-04-18 18:49:08,Samsung,A53 5G,39S,9799.0,2799.0,0.0,...,2023-04-25,NaN,NaN,14,-17.35,-0.14,0.0,0,"J0,G1,E0,E2",0


In [19]:
credi.isnull().sum()

folio                    0
tag                      0
folio_solicitud          0
fecha                    0
marca                    0
modelo                   0
plazo                    0
precio                   0
enganche                 0
descuento                0
semana                   0
monto_financiado         0
costo_total              0
monto_accesorios         0
agente_venta             0
dis_venta                0
status                   0
fraude                   0
empresa                 13
inversion                0
pagos_realizados         0
reautorizacion           0
fecha_ultimo_pago     3356
fecha_pago_proximo    3356
status_cuenta         4197
puntos                3595
riesgo                2473
porc_enganche         2473
porc_tasa                0
score_buro               0
razones_buro          4505
semana_actual         4505
codigo_postal          685
dtype: int64

No puede haber medio fraudes entonces, entonces la mayoría no tienen ni un fraude, por lo quie se llena con la moda

In [20]:
credi['fraude'].mode()

0    0
Name: fraude, dtype: int64

Como son fechas en desorden no predecibles, por lo que se dejan en fechas desconocidas

In [21]:
credi['fecha_ultimo_pago']=credi['fecha_ultimo_pago'].fillna('Desconocido')

In [22]:
credi['fecha_pago_proximo']=credi['fecha_pago_proximo'].fillna('Desconocido')

Los status de cuenta son Al corriente, Atraso, Mora, Fraude y los espacios en blanco significa que no se ha determinado el estatus, por lo que se deja como "Blanco"

In [23]:
credi['status_cuenta']=credi['status_cuenta'].fillna('Blanco')

Los puntos depende de los clientes y sus situaciones, así que lo hice por el promedio redondeado.

In [24]:
credi["puntos"]=credi["puntos"].fillna(round(credi["puntos"].mean(),0))

In [25]:
credi['riesgo'] = credi['riesgo'].replace('Cancelado', 167)
credi['riesgo'] = credi['riesgo'].replace('Fraude', 169)
credi['riesgo'] = credi['riesgo'].replace('Atraso', 171)
credi['riesgo'] = pd.to_numeric(credi['riesgo'])

In [26]:
def categorize_riesgo(parametros):
    if pd.isna(parametros):
        return None
    if parametros <= 0:
        return 'No hay riesgo'
    elif 0 <= parametros <= 51:
        return 'Poco riesgo'
    elif 52 <= parametros <= 99:
        return 'Riesgo considerable'
    elif 100 <= parametros <= 103:
        return 'Hay riesgo alto'
    elif parametros == 167:
        return 'Cancelado'
    elif parametros == 169:
        return 'Fraude'
    elif parametros == 171:
        return 'Atraso'

credi['riesgo_categorizado'] = credi['riesgo'].apply(categorize_riesgo)

In [27]:
credi["porc_enganche"]=credi["porc_enganche"].fillna(round(credi["porc_enganche"].mean(),0))

In [28]:
credi["razones_buro"]=credi["razones_buro"].fillna("Desconocido")

In [29]:
credi["razones_buro"]=credi["razones_buro"].fillna("Desconocido")

In [30]:
credi["semana_actual"]=credi["semana_actual"].fillna("Desconocido")

In [32]:
credi["codigo_postal"]=credi["codigo_postal"].fillna("Desconocido")

In [33]:
credi.to_csv('Propuesta uno')